# Testing Grok for reading tweets

In [14]:
from dotenv import load_dotenv

load_dotenv()

import requests
import os
def call_grok_api(prompt: str) -> str:
    
    """
    Makes an API call to Grok with the given prompt
    Returns the response text
    """
    # TODO: Replace with actual Grok API endpoint and key
    GROK_API_ENDPOINT = "https://api.x.ai/v1/chat/completions" 
    GROK_API_KEY = os.getenv("GROK_API_KEY")

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {GROK_API_KEY}"
    }

    payload = {
        "model": "grok-2-latest", 
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7
    }

    try:
        response = requests.post(
            GROK_API_ENDPOINT,
            headers=headers,
            json=payload
        )
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print(f"Error calling Grok API: {e}")
        return ""




# LangGraph test

In [16]:
from typing import TypedDict, List
from datetime import datetime
from langchain_core.messages import SystemMessage
from langgraph.graph import StateGraph, END, START
from tavily import TavilyClient
import os



# Define state schema
class MasterState(TypedDict):
    messages: List[str]
    search_results: List[dict]
    grok_response: str

# Initialize Tavily client
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

def search_news(state: MasterState):
    """Search for recent news using Tavily"""
    query = state["messages"][-1] # Get last message
    
    # Search with Tavily
    search_results = tavily_client.search(
        query=f"{query} {datetime.now().strftime('%m-%Y')}", 
        topic="news",
        days=3,
        max_results=5
    )
    
    return {"search_results": search_results}

def verify_with_grok(state: MasterState):
    """Have Grok verify the news results"""
    search_results = state["search_results"]
    print("search_results", search_results)
    
    # Format search results into prompt
    results_text = "\n".join([
        f"- {result['title']}: {result['content'][:200]}..." 
        for result in search_results["results"]  # Access the "results" list from search_results
    ])
    
    prompt = f"""Here are some recent news results:
{results_text}

Please verify these news items and provide your analysis of their accuracy and credibility.
Focus on corroborating the key claims and identifying any potential inconsistencies. Provide specific tweets that you used as evidence in your answer"""

    grok_response = call_grok_api(prompt)
    
    return {"grok_response": grok_response}

# Build graph
workflow = StateGraph(MasterState)

# Add nodes

workflow.add_edge(START, "search_news")
workflow.add_node("search_news", search_news)
workflow.add_node("verify_with_grok", verify_with_grok) 

# Add edges
workflow.add_edge("search_news", "verify_with_grok")
workflow.add_edge("verify_with_grok", END)

# Compile
news_verification_chain = workflow.compile()

# Example usage:
initial_state = {
    "messages": ["What are the latest developments in AI?"],
    "search_results": [],
    "grok_response": ""
}

result = news_verification_chain.invoke(initial_state)
print(f"Grok's Analysis:\n{result['grok_response']}")

search_results {'query': 'What are the latest developments in AI? 02-2025', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.biotecnika.org/2025/02/ai-in-biotech-the-future-in-here-are-you-ready/', 'title': 'AI in Biotech: The Future Is Here! Are You Ready? - BioTecNika', 'score': 0.5198388, 'published_date': 'Fri, 14 Feb 2025 12:22:51 GMT', 'content': 'AI in Biotech - The Future in Here! Biotecnika Times Newsletter 13.02.2025 – Pfizer Biotech Internship, Freshers Jobs at… Will AI Take Our Jobs- Should Biotech Candidates Be Worried Home Career Advice AI in Biotech: The Future Is Here! AI in Biotech: The Future Is Here! AI in Biotech: The Future Is Here! In biotechnology, AI has broken the path of traditional coding from advanced algorithms and machine learning to describe complex biological processes so that precise and predictive insights can be made. The Future of AI in Biotechnology – AI in Biotech Insilico Medicine: Uses AI to design new d